# Clean and Prep
With the additional data given to us, we wanted to include it in our training data as to get the best predictions possible for March. In order to do that, we need to clean the data the same way we cleaned the original data. That newly cleaned data will then be concatenated to the original data, replacing the older February dates. As it is so late in the project cycle, this is only being done for the data that our final model is being trained on, to save time and for efficiency purposes.

In [26]:
#data access and processing
import pandas as pd
import numpy as np
import os
from datetime import date, datetime, timedelta

os.chdir('/data/p_dsi/teams2023/team9/')

new = pd.read_csv('Asurion_data_additional.csv')

new = new[new.columns[1:]]

new.head()

,phone model,phone size,phone color,weeks_monday,claim
0,alcatel smartflip 4052r,4gb,black,2/27/2023,5
1,alcatel tetra,16gb,black,2/27/2023,1
2,alcatel volta,16gb,gray,2/27/2023,1
3,apple iphone 11,128gb,black,2/6/2023,205
4,apple iphone 11,128gb,black,2/13/2023,189


### Select just apple data

In [27]:
new["make"] = np.where(new["phone model"].str.startswith("alcatel"), "alcatel",
              np.where(new["phone model"].str.startswith("apple"), "apple",
              np.where(new["phone model"].str.startswith("samsung"), "samsung", 
              np.where(new["phone model"].str.startswith("lg"), "lg",
              np.where(new["phone model"].str.startswith("google"), "google", "other")))))

new_apple = new[new["make"] == "apple"]

new_apple.head()

,phone model,phone size,phone color,weeks_monday,claim,make
3,apple iphone 11,128gb,black,2/6/2023,205,apple
4,apple iphone 11,128gb,black,2/13/2023,189,apple
5,apple iphone 11,128gb,black,2/20/2023,158,apple
6,apple iphone 11,128gb,black,2/27/2023,179,apple
7,apple iphone 11,128gb,green,2/6/2023,43,apple


### Select just recent iPhones

In [28]:
# Subsetting into groups
new_apple["model_group"] = np.where(new_apple["phone model"].str.startswith("apple iphone x"), "iphone x",
                           np.where(new_apple["phone model"].str.startswith("apple iphone 11"), "iphone 11",
                           np.where(new_apple["phone model"].str.startswith("apple iphone 12"), "iphone 12",
                           np.where(new_apple["phone model"].str.startswith("apple iphone 13"), "iphone 13",
                           np.where(new_apple["phone model"].str.startswith("apple iphone 14"), "iphone 14", 
                           np.where(new_apple["phone model"].str.startswith("apple iphone se"), "iphone se", "other"))))))

# Removing any in group "other"
new_apple = new_apple[new_apple["model_group"] != "other"]

# Drop old phone model group
new_apple = new_apple.drop('phone model', axis = 1)
new_apple = new_apple.drop('make', axis = 1)

# Set to datetime
new_apple['weeks_monday'] = pd.to_datetime(new_apple['weeks_monday'].str[:10])

new_apple.head()

/tmp/ipykernel_9105/837173582.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_apple["model_group"] = np.where(new_apple["phone model"].str.startswith("apple iphone x"), "iphone x",


,phone size,phone color,weeks_monday,claim,model_group
3,128gb,black,2023-02-06,205,iphone 11
4,128gb,black,2023-02-13,189,iphone 11
5,128gb,black,2023-02-20,158,iphone 11
6,128gb,black,2023-02-27,179,iphone 11
7,128gb,green,2023-02-06,43,iphone 11


### Clean phone size column

In [29]:
new_apple['phone size'] = new_apple['phone size'].str[:-2].astype(int)
new_apple.head()

,phone size,phone color,weeks_monday,claim,model_group
3,128,black,2023-02-06,205,iphone 11
4,128,black,2023-02-13,189,iphone 11
5,128,black,2023-02-20,158,iphone 11
6,128,black,2023-02-27,179,iphone 11
7,128,green,2023-02-06,43,iphone 11


### Generate time-related columns

In [30]:
new_apple.set_index('weeks_monday')
new_apple['weeks_monday'] = pd.to_datetime(new_apple['weeks_monday'], format='%Y-%m-%d')
new_apple.sort_index()

new_apple['month'] = new_apple['weeks_monday'].dt.month
new_apple['year'] = new_apple['weeks_monday'].dt.year
new_apple['weekofyear'] = new_apple['weeks_monday'].dt.isocalendar().week.astype(int)

new_apple['day'] = new_apple['weeks_monday'].dt.day

new_apple['winter'] = 1
new_apple['spring'] = 0
new_apple['summer'] = 0
new_apple['autumn'] = 0

new_apple['Q1'] = 1
new_apple['Q2'] = 0
new_apple['Q3'] = 0
new_apple['Q4'] = 0

new_apple.rename(columns={"phone color": "phone_color"})

new_apple.head()

,phone size,phone color,weeks_monday,claim,model_group,month,year,weekofyear,day,winter,spring,summer,autumn,Q1,Q2,Q3,Q4
3,128,black,2023-02-06,205,iphone 11,2,2023,6,6,1,0,0,0,1,0,0,0
4,128,black,2023-02-13,189,iphone 11,2,2023,7,13,1,0,0,0,1,0,0,0
5,128,black,2023-02-20,158,iphone 11,2,2023,8,20,1,0,0,0,1,0,0,0
6,128,black,2023-02-27,179,iphone 11,2,2023,9,27,1,0,0,0,1,0,0,0
7,128,green,2023-02-06,43,iphone 11,2,2023,6,6,1,0,0,0,1,0,0,0


### Generate one-hot encode columns

In [31]:
#one-hot encoder for categorical variables
one_hot_encoded = pd.get_dummies(new_apple['phone color'])
new_apple = new_apple.join(one_hot_encoded)
new_apple = new_apple.drop('phone color', axis = 1)

one_hot_encoded = pd.get_dummies(new_apple['model_group'])
new_apple = new_apple.join(one_hot_encoded)
new_apple = new_apple.drop('model_group', axis = 1)

new_apple.head()

,phone size,weeks_monday,claim,month,year,weekofyear,day,winter,spring,summer,...,silver,starlight,white,yellow,iphone 11,iphone 12,iphone 13,iphone 14,iphone se,iphone x
3,128,2023-02-06,205,2,2023,6,6,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,128,2023-02-13,189,2,2023,7,13,1,0,0,...,0,0,0,0,1,0,0,0,0,0
5,128,2023-02-20,158,2,2023,8,20,1,0,0,...,0,0,0,0,1,0,0,0,0,0
6,128,2023-02-27,179,2,2023,9,27,1,0,0,...,0,0,0,0,1,0,0,0,0,0
7,128,2023-02-06,43,2,2023,6,6,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [32]:
# Removing colors not in original data
new_apple = new_apple.drop(['(universal) gray', '(universal) silver'], axis=1)

print(new_apple.columns)

Index(['phone size', 'weeks_monday', 'claim', 'month', 'year', 'weekofyear',
       'day', 'winter', 'spring', 'summer', 'autumn', 'Q1', 'Q2', 'Q3', 'Q4',
       'black', 'blue', 'coral', 'gold', 'graphite', 'gray', 'green',
       'midnight', 'pink', 'purple', 'red', 'silver', 'starlight', 'white',
       'yellow', 'iphone 11', 'iphone 12', 'iphone 13', 'iphone 14',
       'iphone se', 'iphone x'],
      dtype='object')


In [33]:
# Finally, remove weeks_monday to line up with original data
new_apple = new_apple.drop(['weeks_monday'], axis = 1)
new_apple.head()

,phone size,claim,month,year,weekofyear,day,winter,spring,summer,autumn,...,silver,starlight,white,yellow,iphone 11,iphone 12,iphone 13,iphone 14,iphone se,iphone x
3,128,205,2,2023,6,6,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,128,189,2,2023,7,13,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,128,158,2,2023,8,20,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6,128,179,2,2023,9,27,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
7,128,43,2,2023,6,6,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


## Load in the older data

In [34]:
# yayaya
old = pd.read_csv('output.csv')

old.head()

,phone size,claim,year,black,blue,coral,gold,graphite,gray,green,...,Q4,iphone 11,iphone 12,iphone 13,iphone 14,iphone se,iphone x,weekofyear,month,day
0,128,252,2022,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,4,1,24
1,128,45,2022,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,4,1,24
2,128,92,2022,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,4,1,24
3,128,69,2022,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,4,1,24
4,128,91,2022,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,4,1,24


In [35]:
old.loc[np.invert((old['year'] == 2023) & (old['month'] == 2))]

,phone size,claim,year,black,blue,coral,gold,graphite,gray,green,...,Q4,iphone 11,iphone 12,iphone 13,iphone 14,iphone se,iphone x,weekofyear,month,day
0,128,252,2022,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,4,1,24
1,128,45,2022,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,4,1,24
2,128,92,2022,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,4,1,24
3,128,69,2022,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,4,1,24
4,128,91,2022,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,4,1,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11368,512,11,2023,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,5,1,30
11369,512,1,2023,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,5,1,30
11370,64,34,2023,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,5,1,30
11371,64,38,2023,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,5,1,30


### Remove current february rows that will be replaced with new data

In [36]:
old_trim = old.loc[np.invert((old['year'] == 2023) & (old['month'] == 2))]
old_trim.shape

(11373, 35)

In [37]:
new_apple.shape

(691, 35)

### Concatenate the data

In [38]:
final_data = pd.concat([old_trim, new_apple], axis=0)
final_data.shape

(12064, 35)

In [40]:
final_data['year'].value_counts()

2022    7836
2021    2693
2023    1535
Name: year, dtype: int64

In [43]:
final_data = final_data.sort_values(['year', 'weekofyear'], ascending = [True, True])

### Output the data

In [46]:
# only run 1 time
# final_data.to_csv('final_input_data.csv', index = False)